# Tool 0 - Business Request Parser Demo

**Purpose:** Parse standardized Markdown business documents into structured JSON using LangGraph.

**Acceptance Criteria:**
- ✅ Load sample Markdown document
- ✅ Parse via LangGraph structured output (Pydantic schema)
- ✅ Display JSON under cell
- ✅ Save result + prompt to `data/tool0_samples/`
- ✅ Inline implementation (v1) for testing; module in `src/tool0/` exists for future reuse

**Note:** This implements MVP version - single LLM call without regex post-processing.

In [1]:
# Install required packages (run once)
# !pip install langgraph langchain langchain-openai langchain-anthropic pydantic python-dotenv

In [2]:
# Import required modules
from pydantic import BaseModel, Field, field_validator
from datetime import datetime
import json
from pathlib import Path

# Define Pydantic schemas inline
class ProjectMetadata(BaseModel):
    """Metadata about the business project request."""

    project_name: str = Field(
        description="Name of the project"
    )
    sponsor: str = Field(
        description="Name of the project sponsor"
    )
    submitted_at: str = Field(
        description="Date when the request was submitted, in ISO 8601 format (YYYY-MM-DD)"
    )
    extra: dict[str, str] = Field(
        default_factory=dict,
        description="Additional metadata fields as key-value pairs"
    )

    @field_validator('submitted_at')
    @classmethod
    def validate_iso_date(cls, v: str) -> str:
        """Validate that date is in ISO 8601 format."""
        try:
            datetime.fromisoformat(v)
            return v
        except ValueError:
            raise ValueError(f"Date must be in ISO 8601 format (YYYY-MM-DD), got: {v}")


class BusinessRequest(BaseModel):
    """Structured representation of a parsed business request document."""

    project_metadata: ProjectMetadata = Field(
        description="Project metadata including name, sponsor, and submission date"
    )
    goal: str = Field(
        default="unknown",
        description="Main goal or objective of the project"
    )
    scope_in: str = Field(
        default="unknown",
        description="What is included in the project scope"
    )
    scope_out: str = Field(
        default="unknown",
        description="What is explicitly excluded from the project scope"
    )
    entities: list[str] = Field(
        default_factory=list,
        description="Key business entities involved in the project"
    )
    metrics: list[str] = Field(
        default_factory=list,
        description="Key metrics or KPIs to be tracked"
    )
    sources: list[str] = Field(
        default_factory=list,
        description="Expected data sources for the project"
    )
    constraints: list[str] = Field(
        default_factory=list,
        description="Constraints, limitations, or special requirements"
    )
    deliverables: list[str] = Field(
        default_factory=list,
        description="Required deliverables or artifacts from the project"
    )

print("✅ Schemas defined successfully")

✅ Schemas defined successfully


## 1. Load Sample Business Document

We'll use the sample document in `data/sample_business_request.md`

In [3]:
# Hardcoded sample business document
business_document = """# Požadavek na datový projekt

## Projekt
**Název:** BA/BS Supplier Analytics Platform
**Sponzor:** Petra  Nováková
**Datum:** 2025-10-25
**Oddělení:** Business Analytics
**Priorita:** Vysoká

## Cíl
Vytvořit komplexní analytickou platformu pro reporting a analýzu dodavatelských dat v BA/BS datamartů. Platforma umožní business uživatelům sledovat výkonnost dodavatelů, identifikovat rizikové partnery a optimalizovat nákupní procesy na základě historických dat.

## Rozsah

### In Scope
- Analýza dodavatelů (suppliers) a jejich výkonnosti
- Reporting nákupních objednávek (purchase orders)
- Sledování kvality dodávek (delivery quality metrics)
- Dimenzionální model pro dodavatelské údaje
- Integrace s existujícími BA/BS datamartami

### Out of Scope
- HR data o zaměstnancích
- Finanční forecasting a budgetování
- Real-time monitoring dodávek
- Integrace s externí CRM systémy

## Klíčové entity & metriky

### Entity
- Suppliers (dodavatelé)
- Purchase Orders (nákupní objednávky)
- Products (produkty)
- Delivery Performance (výkonnost dodávek)

### Metriky
- On-time delivery rate (% včasných dodávek)
- Supplier reliability score (hodnocení spolehlivosti)
- Average order value (průměrná hodnota objednávky)
- Quality defect rate (% vadných dodávek)
- Lead time průměr

## Očekávané zdroje
- Databricks Unity Catalog (BA/BS datamart schemas)
- Collibra Data Catalog (metadata governance)
- SAP Tables (zdrojové transakční data)
- Historical supplier performance logs

## Omezení
- GDPR compliance - žádné osobní údaje bez souhlasu
- Data retention max 5 let
- Maximální response time pro dashboardy: 3 sekundy
- Read-only přístup k produkčním datám
- Row Level Security podle business unit

## Požadované artefakty
- ER diagram v Mermaid formátu
- Power Query M skripty pro data refresh
- Governance report (kvalita metadat, validace)
- Security report (RLS návrh, klasifikace)
- Dokumentace datového modelu
"""

print(f"📄 Business document loaded ({len(business_document)} characters)")
print("\nFirst 300 characters:")
print("=" * 60)
print(business_document[:300])
print("...")

📄 Business document loaded (1919 characters)

First 300 characters:
# Požadavek na datový projekt

## Projekt
**Název:** BA/BS Supplier Analytics Platform
**Sponzor:** Petra  Nováková
**Datum:** 2025-10-25
**Oddělení:** Business Analytics
**Priorita:** Vysoká

## Cíl
Vytvořit komplexní analytickou platformu pro reporting a analýzu dodavatelských dat v BA/BS datamart
...


## 2. Parse Document Using LangGraph

Call `parse_business_request()` which uses LangGraph with structured output.

In [4]:
# Parse the business document using LangGraph
from langchain.agents import create_agent

print("🔄 Parsing document with LangGraph...")

# System prompt for parsing
SYSTEM_PROMPT = """You are a business requirements parser. Your task is to extract structured information from business request documents.

Documents may contain a mix of Czech and English. Common section headers include:
- "Projekt" / "Project" - project metadata (name, sponsor, date)
- "Cíl" / "Goal" - main project objective
- "Rozsah" / "Scope" - what is in/out of scope
- "Klíčové entity & metriky" / "Key entities & metrics" - business entities and KPIs
- "Očekávané zdroje" / "Expected sources" - data sources
- "Omezení" / "Constraints" - limitations and requirements
- "Požadované artefakty" / "Required artifacts" - deliverables

IMPORTANT INSTRUCTIONS:
1. Extract information into the structured format exactly as specified
2. Use "unknown" for any missing sections
3. Ensure dates are in ISO 8601 format (YYYY-MM-DD)
4. Extract lists as arrays of strings, not concatenated text
5. For project metadata, look for project name, sponsor name, and submission date
6. Any additional metadata fields should go into the "extra" dictionary
7. Be thorough - extract all relevant information from the document
"""

# Create agent with structured output
agent = create_agent(
    model="openai:gpt-5-mini",
    response_format=BusinessRequest,  # Auto-selects best strategy
    system_prompt=SYSTEM_PROMPT
)

# Prepare user message
user_message = f"""Parse the following business request document:

{business_document}

Extract all information into the structured format."""

# Invoke agent
result = agent.invoke({
    "messages": [
        {"role": "user", "content": user_message}
    ]
})

# Extract structured response
structured_response = result.get("structured_response")

if not structured_response:
    raise ValueError("No structured response returned from agent")

# Convert to dict
parsed_json = structured_response.model_dump() if hasattr(structured_response, 'model_dump') else structured_response.dict()

# Extract raw response
raw_response = str(result.get("messages", [])[-1] if result.get("messages") else structured_response)

# Full prompt for audit
prompt_used = f"System: {SYSTEM_PROMPT}\n\nUser: {user_message}"

print("✅ Parsing complete!")

🔄 Parsing document with LangGraph...
✅ Parsing complete!


## 3. Display Parsed JSON

Show the structured output directly under this cell.

In [5]:
# Display parsed JSON
print("📊 Parsed Business Request:")
print("=" * 60)
print(json.dumps(parsed_json, indent=2, ensure_ascii=False))

# Also show as Pydantic model
print("\n" + "=" * 60)
print("📋 Validation:")
try:
    validated = BusinessRequest.model_validate(parsed_json)
    print(f"✅ Schema valid: {validated.project_metadata.project_name}")
    print(f"   Sponsor: {validated.project_metadata.sponsor}")
    print(f"   Date: {validated.project_metadata.submitted_at}")
    print(f"   Entities: {len(validated.entities)} found")
    print(f"   Sources: {len(validated.sources)} found")
except Exception as e:
    print(f"❌ Validation error: {e}")

📊 Parsed Business Request:
{
  "project_metadata": {
    "project_name": "BA/BS Supplier Analytics Platform",
    "sponsor": "Petra Nováková",
    "submitted_at": "2025-10-25",
    "extra": {
      "department": "Business Analytics",
      "priority": "Vysoká"
    }
  },
  "goal": "Vytvořit komplexní analytickou platformu pro reporting a analýzu dodavatelských dat v BA/BS datamartů. Platforma umožní business uživatelům sledovat výkonnost dodavatelů, identifikovat rizikové partnery a optimalizovat nákupní procesy na základě historických dat.",
  "scope_in": "Analýza dodavatelů (suppliers) a jejich výkonnosti; Reporting nákupních objednávek (purchase orders); Sledování kvality dodávek (delivery quality metrics); Dimenzionální model pro dodavatelské údaje; Integrace s existujícími BA/BS datamartami",
  "scope_out": "HR data o zaměstnancích; Finanční forecasting a budgetování; Real-time monitoring dodávek; Integrace s externí CRM systémy",
  "entities": [
    "Suppliers (dodavatelé)",
    

## 4. Save Results to data/tool0_samples/

Save both JSON result and prompt for regression testing.

In [6]:
# Save results to data/tool0_samples/
timestamp = datetime.now().isoformat()
output_dir = Path.cwd().parent / 'data' / 'tool0_samples'
output_dir.mkdir(parents=True, exist_ok=True)

# Save JSON result
json_path = output_dir / f"{timestamp}.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(parsed_json, f, indent=2, ensure_ascii=False)

# Save prompt
md_path = output_dir / f"{timestamp}.md"
with open(md_path, 'w', encoding='utf-8') as f:
    f.write(f"# Parse Request - {timestamp}\n\n")
    f.write(f"## Prompt Used\n\n```\n{prompt_used}\n```\n\n")
    f.write(f"## Raw Response\n\n```\n{raw_response}\n```\n\n")
    f.write(f"## Parsed JSON\n\n```json\n{json.dumps(parsed_json, indent=2, ensure_ascii=False)}\n```\n")

print(f"💾 Results saved:")
print(f"   JSON: {json_path}")
print(f"   Markdown: {md_path}")

💾 Results saved:
   JSON: /Users/marekminarovic/archi-agent/data/tool0_samples/2025-10-31T01:14:27.960789.json
   Markdown: /Users/marekminarovic/archi-agent/data/tool0_samples/2025-10-31T01:14:27.960789.md


## 5. Summary

✅ **Acceptance Criteria Met (v1 - Inline Approach):**
- [x] Jupyter notebook with sample business document (hardcoded)
- [x] Single LLM call (no regex) converts to valid JSON
- [x] Structured output via Pydantic schema (BusinessRequest)
- [x] JSON displayed under cell
- [x] Results saved to `data/tool0_samples/` (JSON + Markdown)
- [x] Inline implementation (no external imports for v1 testing)

**Implementation Details:**
- **Schemas:** Defined inline in Cell 3 (ProjectMetadata, BusinessRequest)
- **Document:** Hardcoded in Cell 5 (no file I/O)
- **Parser:** Inline LangGraph agent creation in Cell 7
- **Model:** OpenAI GPT-5-mini with structured output
- **Output:** parsed_json, raw_response, prompt_used for audit trail

**Next Steps:**
- Run compliance checker: `python3 .claude/skills/langchain/compliance-checker/check.py --file src/tool0/parser.py`
- Update story frontmatter: `skill_created: true`, `skill_status: ready_to_execute`
- Refactor to modular structure (optional - use src/tool0/parser.py after v1 validation)